In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.chdir("/work/users/minhnth/projects/GIP/")

In [10]:
# Read a vcf file
def read_vcf(vcf_path):
    with open(vcf_path, "rt") as ifile:
          for line in ifile:
            if line.startswith("#CHROM"):
                  vcf_names = [x for x in line.split('\t')]
                  break
    ifile.close()
    data = pd.read_csv(vcf_path, comment='#', sep="\s+", header=None, names=vcf_names)
    return data
def get_data(X, miss_rate):
    # Parameters
    no, dim = X.shape

    # Introduce missing data
    data_m = binary_sampler(1-miss_rate, no, dim)
    miss_data_x = X.copy()
    miss_data_x[data_m == 0] = ".|."
    return X, miss_data_x, data_m
def binary_sampler(p, rows, cols):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  np.random.seed(7)
  unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [4]:
# Load original data
vcf_path =  "data/HLA/HLA1_chr6.vcf"
rate = 0.2
data = read_vcf(vcf_path)
geno = data.iloc[:, 9::]
geno.rename(columns={'NA21144\n':'NA21144'}, inplace=True)

In [5]:
geno.head()

,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,HG00107,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0
2,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [6]:
meta_data = pd.read_csv("data/HLA/igsr-1000 genomes on grch38.tsv", sep = "\t")

In [7]:
np.unique(list(meta_data["Superpopulation code"]))

array(['AFR', 'AMR', 'EAS', 'EUR', 'EUR,AFR', 'SAS'], dtype='<U7')

In [13]:
AFR = meta_data["Sample name"][meta_data["Superpopulation code"] == 'AFR']
AMR = meta_data["Sample name"][meta_data["Superpopulation code"] == 'AMR']
EAS = meta_data["Sample name"][meta_data["Superpopulation code"] == 'EAS']
EUR = meta_data["Sample name"][meta_data["Superpopulation code"] == 'EUR']
SAS = meta_data["Sample name"][meta_data["Superpopulation code"] == 'SAS']

In [17]:
pop = geno.columns
AFR_inter = set(AFR).intersection(set(pop)) 
geno_afr = geno.loc[:, list(AFR_inter)]

In [23]:
geno_afr

,HG03476,HG03175,HG02308,HG02611,NA19324,NA18868,NA19461,NA18876,NA18933,HG02489,...,HG02814,NA19445,NA20359,NA20281,HG03469,NA19908,HG03111,NA19327,HG03114,HG02561
0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1054,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1055,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1056,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [9]:
#geno, geno_miss, m = get_data(geno, rate)
#gn = pd.concat([data.iloc[:, 0:9], geno_miss], axis = 1)